In [5]:
from graph import Graph
import networkx as nx
from scipy.sparse.linalg import eigs
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import scipy

In [3]:
def get_eigens(matrix, k, graph_name, graph_type):
    vals, vecs = eigs(matrix.asfptype(), k=k, sigma=0, OPpart='r')
    filename = "k_"+str(k)+"_"+graph_name + "_" + graph_type + ".npy"
    np.save(filename,vecs)
    return vecs

def get_generalized_eigens(matrix, degree, k, graph_name, graph_type):
    vals, vecs = eigs(matrix.asfptype(), k=k, M=degree, sigma=0, OPpart='r')
    filename = "k_"+str(k)+"_"+graph_name + "_" + graph_type + ".npy"
    np.save(filename,vecs)
    return vecs

In [6]:
def save_eigens(fnames_competition, k_s):
    for i, fname in enumerate(fnames_competition):
        graph = Graph(fname=fname,fpath="")
        laplacian = nx.laplacian_matrix(graph.G)
        get_eigens(laplacian.asfptype(), k_s[i], fname, "laplacian")
        
def save_normalized_eigens(fnames_competition, k_s):
    for i, fname in enumerate(fnames_competition):
        graph = Graph(fname=fname,fpath="")
        laplacian = nx.normalized_laplacian_matrix(graph.G)
        get_eigens(laplacian.asfptype(), k_s[i], fname, "normalized_laplacian")
        
def save_generalized_eigens(fnames_competition, k_s):
    for i, fname in enumerate(fnames_competition):
        graph = Graph(fname=fname,fpath="")
        laplacian = nx.laplacian_matrix(graph.G)
        degree = scipy.sparse.diags(np.array(graph.G.degree())[:,1])
        get_generalized_eigens(laplacian.asfptype(), degree.asfptype(), k_s[i], fname, "generalized_eigenproblem")
        
        
fnames_competition = ['ca-GrQc',
                      'Oregon-1',
                      'soc-Epinions1',
                      'web-NotreDame',
                      'roadNet-CA'
                      ]
k_s = [100,100,100,100,100]

#save_eigens(fnames_competition, k_s)
#save_normalized_eigens(fnames_competition, k_s)
save_generalized_eigens(fnames_competition, k_s)

Reading edge data from /Users/juliushietala/Desktop/School/amdm/amdm-project-2019/graphs_processed/ca-GrQc.txt
   node1  node2
0      0   4016
1      1   2977
2      1    258
3      1    421
4      1   1201
Added nodes to graph
Added edges to graph


/Users/juliushietala/anaconda3/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:295: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Reading edge data from /Users/juliushietala/Desktop/School/amdm/amdm-project-2019/graphs_processed/Oregon-1.txt
   node1  node2
0      0    932
1      1    932
2      1    570
3      1   1620
4      2   2211
Added nodes to graph
Added edges to graph
Reading edge data from /Users/juliushietala/Desktop/School/amdm/amdm-project-2019/graphs_processed/soc-Epinions1.txt
   node1  node2
0      0      1
1      0      2
2      0      3
3      0      4
4      0      5
Added nodes to graph
Added edges to graph
Reading edge data from /Users/juliushietala/Desktop/School/amdm/amdm-project-2019/graphs_processed/web-NotreDame.txt
   node1  node2
0      0      0
1      0      1
2      0      2
3      0      3
4      0      4
Added nodes to graph
Added edges to graph
Reading edge data from /Users/juliushietala/Desktop/School/amdm/amdm-project-2019/graphs_processed/roadNet-CA.txt
   node1  node2
0      0      1
1      0      2
2      0    466
3      1      6
4      1    382
Added nodes to graph
Added edg

In [68]:
def test_laplacian(graph_name, k):
    eigs = np.load('./eigenvectors/laplacian/'+graph_name+'_laplacian.npy')
    labels = KMeans(init='k-means++', n_clusters=k).fit_predict(eigs)
    graph = Graph(fname=graph_name, fpath="")
    total_conductance = 0
    for i in range(k):
        idx = np.where(labels == i)[0]
        print("Length of cluster:", len(idx))
        conductance = nx.algorithms.cuts.cut_size(graph.G, idx) / len(idx)
        total_conductance += conductance
        print("Conductance of cluster", i, ":", conductance)
    print("Total conductance", total_conductance)
    
def test_generalized(graph_name, k):
    eigs = np.load('./eigenvectors/generalized_eigenproblem/'+graph_name+'_generalized_eigenproblem.npy')
    labels = KMeans(init='k-means++', n_clusters=k).fit_predict(eigs)
    graph = Graph(fname=graph_name, fpath="")
    total_conductance = 0
    for i in range(k):
        idx = np.where(labels == i)[0]
        print("Length of cluster:", len(idx))
        conductance = nx.algorithms.cuts.cut_size(graph.G, idx) / len(idx)
        total_conductance += conductance
        print("Conductance of cluster", i, ":", conductance)
    print("Total conductance", total_conductance)
    
def test_normalized(graph_name, k):
    eigs = np.load('./eigenvectors/normalized_laplacian/'+graph_name+'_normalized_laplacian.npy')
    eigs = eigs/np.linalg.norm(eigs, ord=2, axis=1, keepdims=True)
    labels = KMeans(init='k-means++', n_clusters=k).fit_predict(eigs)
    graph = Graph(fname=graph_name, fpath="")
    total_conductance = 0
    for i in range(k):
        idx = np.where(labels == i)[0]
        print("Length of cluster:", len(idx))
        conductance = nx.algorithms.cuts.cut_size(graph.G, idx) / len(idx)
        total_conductance += conductance
        print("Conductance of cluster", i, ":", conductance)
    print("Total conductance", total_conductance)

In [74]:
fnames_competition = ['ca-GrQc',
                      'Oregon-1',
                      'soc-Epinions1',
                      'web-NotreDame',
                      'roadNet-CA'
                      ]
k_s = [2,5,10,20,50]

graph_name =fnames_competition[3]
k = k_s[3]

test_laplacian(graph_name, k)
test_normalized(graph_name, k)
test_generalized(graph_name, k)

/Users/juliushietala/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


Reading edge data from /Users/juliushietala/Desktop/School/amdm/amdm-project-2019/graphs_processed/web-NotreDame.txt
   node1  node2
0      0      0
1      0      1
2      0      2
3      0      3
4      0      4
Added nodes to graph
Added edges to graph
Length of cluster: 265039
Conductance of cluster 0 : 0.00021506268888729584
Length of cluster: 1754
Conductance of cluster 1 : 0.0017103762827822121
Length of cluster: 2625
Conductance of cluster 2 : 0.001142857142857143
Length of cluster: 2660
Conductance of cluster 3 : 0.00037593984962406017
Length of cluster: 2652
Conductance of cluster 4 : 0.0003770739064856712
Length of cluster: 2938
Conductance of cluster 5 : 0.0010211027910142954
Length of cluster: 501
Conductance of cluster 6 : 0.001996007984031936
Length of cluster: 10740
Conductance of cluster 7 : 0.0008379888268156424
Length of cluster: 492
Conductance of cluster 8 : 0.0020325203252032522
Length of cluster: 501
Conductance of cluster 9 : 0.001996007984031936
Length of cluste

In [ ]:
from graph import Graph
import networkx as nx
from scipy.sparse.linalg import eigs

graph = Graph(fname='roadNet-CA')
laplacian = nx.laplacian_matrix(graph.G)
vals, vecs = eigs(laplacian.asfptype(), k=2, sigma=0, OPpart='r')